<center>
    <p style="text-align:center">
    <img alt="arize logo" src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="300"/>
        <br>
        <a href="https://docs.arize.com/arize/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/client_python">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-11t1vbu4x-xkBIHmOREQnYnYDH1GDfCg">Slack Community</a>
    </p>
</center>



# Azure AI Red Teaming Agent + Arize Tracing

This notebook walks through how to trace Azure AI red teaming agent scans


Prerequisites:

1. Arize AX account ([Sign up for free](https://app.arize.com/auth/join))
2. Azure AI foundry account and project created  ([Sign up here](https://azure.microsoft.com/en-us/products/ai-foundry))

Information on AI Red Teaming agent: https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/ai-red-teaming-agent?source=recommendations


## Prerequisites

- Python 3.10-3.12
- Azure AI Foundry project
- OpenAI API key (or Azure OpenAI endpoint)
- Required packages installed

In [ ]:
# Install required packages
!pip install -q "azure-ai-evaluation[redteam]" openai arize-otel openinference-instrumentation-openai

## 1. Create the Azure AI Red Teaming Agent

You can instantiate the AI Red Teaming agent with your Azure AI Project and Azure Credentials.

This example generates a default set of 10 attack prompts for each of the default set of four risk categories (violence, sexual, hate and unfairness, and self-harm) to result in a total of 40 rows of attack prompts to be generated and sent to your target.

https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/develop/run-scans-ai-red-teaming-agent#results-from-your-automated-scans



In [ ]:
# Setup up red teaming agent
import os

# Azure imports
from azure.identity import DefaultAzureCredential
from azure.ai.evaluation.red_team import RedTeam, RiskCategory, AttackStrategy

#Set up environment variables
os.environ["AZURE_SUBSCRIPTION_ID"] = ""
os.environ["AZURE_RESOURCE_GROUP"] = ""
os.environ["AZURE_PROJECT_NAME"] = ""
os.environ["PROJECT_ENDPOINT"] = ""
os.environ["ARIZE_SPACE_ID"] = ""
os.environ["ARIZE_API_KEY"] = ""
os.environ["PROJECT_NAME"] = "red-team-violence-examples"


## Using Azure AI Foundry Hub project
azure_ai_project = {
    "subscription_id": os.environ["AZURE_SUBSCRIPTION_ID"],
    "resource_group_name": os.environ["AZURE_RESOURCE_GROUP"],
    "project_name": os.environ["AZURE_PROJECT_NAME"],
}
azure_ai_project = os.environ["PROJECT_ENDPOINT"]

# Instantiate your AI Red Teaming Agent
red_team_agent = RedTeam(
    azure_ai_project=azure_ai_project, # required
    credential=DefaultAzureCredential() # required
)

Optionally, you can specify which risk categories of content risks you want to cover with risk_categories parameter and define the number of prompts covering each risk category with num_objectives parameter.

In [ ]:
# Specifying risk categories and number of attack objectives per risk categories you want the AI Red Teaming Agent to cover
red_team_agent = RedTeam(
    azure_ai_project=azure_ai_project, # required
    credential=DefaultAzureCredential(), # required
    risk_categories=[ # optional, defaults to all four risk categories
        RiskCategory.Violence,
        RiskCategory.HateUnfairness,
        RiskCategory.Sexual,
        RiskCategory.SelfHarm
    ], 
    num_objectives=10, # optional, defaults to 10
)

## 2. Trace the Agent

In [ ]:
#enable tracing for openai
from arize.otel import register

tracer_provider = register(
    space_id = os.environ["ARIZE_SPACE_ID"],
    api_key = os.environ["ARIZE_API_KEY"],
    project_name = os.environ["PROJECT_NAME"], 
)

from openinference.instrumentation.openai import OpenAIInstrumentor
OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)

## 3. Create a target for the red teaming agent

In [ ]:
# Set up a callback function to pass to the red teaming agent scan
import openai

# Define a simple callback function that simulates a chatbot
def simple_callback(query: str) -> str:
    # Insert your LLM or agent here
    openai_client = openai.OpenAI()
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system",
            "content": "You are a helpful AI assistant. Always maintain a polite and professional tone. Provide concise answers."
            },
            {"role": "user", "content": query}
        ],
        max_tokens=100,
    )

## 4. Run the red teaming scan

When the scan is finished, you can specify an output_path to capture a JSON file that represents a scorecard of your results for using in your own reporting tool or compliance platform.



In [ ]:

red_team_result = await red_team_agent.scan(target=simple_callback)

###  Additional Examples

### Available Attack Strategies

To discover available attack strategies in your environment:

In [ ]:
# List available AttackStrategy values
print("Available AttackStrategy enum values:")
for attr in dir(AttackStrategy):
    if not attr.startswith('_') and attr.isupper():
        try:
            value = getattr(AttackStrategy, attr)
            print(f"  - AttackStrategy.{attr}")
        except:
            pass

## 5. Now view the traces from your red teaming scans in Arize!